## Задача
Для датасета с отзывами с маркетплейса на русском языке (https://github.com/sismetanin/rureviews) построить модель для предсказания тональности текста.
1. Использовать как минимум 3 модели машинного обучения, решающие задачу классификации (количество классов определить по анализу датасета)
2. Предсказать тональность при помощи NLP-моделей (deeppavlov, natasha и т.д.) 
3. Определить метрики качества моделей и сравнить полученные результаты, в ячейке markdown представить выводы.

In [1]:
!pip install deeppavlov
!pip install fasttext

In [2]:
import pandas as pd
import os
from deeppavlov import build_model # Импортируем функцию build_model из DeepPavlov
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB

data = pd.read_csv('datasets/women-clothing-accessories.3-class.balanced.csv', encoding="utf-8", engine='python', sep='\t')
# Проверяем данные
print(data.head())

# Разделяем данные на признаки и целевую переменную
X = data['review']
y = data['sentiment']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Преобразование текста в числовой формат с помощью TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 1. Логистическая регрессия
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_tfidf, y_train)
y_pred_log_reg = log_reg.predict(X_test_tfidf)
print("Логистическая регрессия:")
print(classification_report(y_test, y_pred_log_reg))
f1_log_reg = f1_score(y_test, y_pred_log_reg, average='weighted')
print(f"F1-метрика (Логистическая регрессия): {f1_log_reg:.4f}")

# 2. Наивный Байес (Multinomial Naive Bayes)
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
y_pred_nb = nb.predict(X_test_tfidf)
print("\nНаивный Байес:")
print(classification_report(y_test, y_pred_nb))
f1_nb = f1_score(y_test, y_pred_nb, average='weighted')
print(f"F1-метрика (Наивный Байес): {f1_nb:.4f}")

# 3. Случайный лес
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train_tfidf, y_train)
y_pred_rf = random_forest.predict(X_test_tfidf)
print("\nСлучайный лес:")
print(classification_report(y_test, y_pred_rf))
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
print(f"F1-метрика (Случайный лес): {f1_rf:.4f}")

# 4. Оцениваем качество модели sentiment_twitter в DeepPavlov
# Фильтруем тестовые данные, исключая третий класс (sentiment_twitter предсказывает только Positive и Negative)
X_test_filtered = X_test[y_test != 'neautral'].str.capitalize()
y_test_filtered = y_test[y_test != 'neautral'].str.capitalize()
# Используем модель DeepPavlov для предсказания
model = build_model('sentiment_twitter')
y_pred_deeppavlov = model.batched_call(X_test_filtered.tolist(), batch_size=1024)
# Оценка качества модели DeepPavlov
print("\nDeepPavlov sentiment_twitter:")
print(classification_report(y_test_filtered, y_pred_deeppavlov))
f1_deeppavlov = f1_score(y_test_filtered, y_pred_deeppavlov, average='weighted')
print(f"F1-метрика DeepPavlov(sentiment_twitter): {f1_deeppavlov:.4f}")

# Сравнение F1-метрик
print("\nСравнение F1-метрик:")
print(f"Логистическая регрессия: {f1_log_reg:.4f}")
print(f"Наивный Байес: {f1_nb:.4f}")
print(f"Случайный лес: {f1_rf:.4f}")
print(f"DeepPavlov(sentiment_twitter): {f1_deeppavlov:.4f}")

                                              review sentiment
0  качество плохое пошив ужасный (горловина напер...  negative
1  Товар отдали другому человеку, я не получила п...  negative
2  Ужасная синтетика! Тонкая, ничего общего с пре...  negative
3  товар не пришел, продавец продлил защиту без м...  negative
4      Кофточка голая синтетика, носить не возможно.  negative
Логистическая регрессия:
              precision    recall  f1-score   support

    neautral       0.64      0.67      0.65      6000
    negative       0.74      0.71      0.73      6000
    positive       0.88      0.87      0.87      6000

    accuracy                           0.75     18000
   macro avg       0.75      0.75      0.75     18000
weighted avg       0.75      0.75      0.75     18000

F1-метрика (Логистическая регрессия): 0.7492

Наивный Байес:
              precision    recall  f1-score   support

    neautral       0.60      0.72      0.65      6000
    negative       0.75      0.63      0.68   

2025-03-14 05:48:50.797 WARNING in 'deeppavlov.core.models.torch_model'['torch_model'] at line 96: Unable to place component TorchTextClassificationModel on GPU, since no CUDA GPUs are available. Using CPU.



DeepPavlov sentiment_twitter:
              precision    recall  f1-score   support

    Negative       0.56      0.92      0.69      6000
    Positive       0.77      0.27      0.40      6000

    accuracy                           0.59     12000
   macro avg       0.66      0.59      0.54     12000
weighted avg       0.66      0.59      0.54     12000

F1-метрика DeepPavlov(sentiment_twitter): 0.5447

Сравнение F1-метрик:
Логистическая регрессия: 0.7492
Наивный Байес: 0.7330
Случайный лес: 0.7214
DeepPavlov(sentiment_twitter): 0.5447


## Итог:
1. Логистическая регрессия показала наилучший результат среди выбранных моделей, достигнув F1-метрики 0.7492.
2. Модель sentiment_twitter из DeepPavlov показала наихудший результат (F1-метрика 0.5447). Это может быть связано с тем, что данная модель была обучена на другом наборе данных (на твитах), и её предсказания оказались не столь точными при применении к отзывам. Кроме того, эта модель обрабатывала только два класса (Positive и Negative), что также могло повлиять на её точность.

## Вывод:
Логистическая регрессия показала лучшие результаты и является наиболее подходящим алгоритмом для классификации отзывов. Модель sentiment_twitter, несмотря на её популярность для анализа тональности, оказалась наименее эффективной в данном контексте, что подчёркивает важность выбора модели, адаптированной к специфике данных.